# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they  in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
import numpy as np

In [2]:
def min_max(x):
    x = float(x)
    return (x-x.min())/(x.min()-x.max())

In [3]:
data = pd.read_csv('census_data.csv')
census = data.copy()
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
census.info()
categorical_col = census.select_dtypes(['O']).columns.values
columns = census.columns.values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
age               32561 non-null int64
workclass         32561 non-null object
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
gender            32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
income_bracket    32561 non-null object
dtypes: int64(5), object(9)
memory usage: 3.5+ MB


In [5]:
for feat in categorical_col:
    le = LabelEncoder()
    census[feat] = le.fit_transform(census[feat])

In [6]:
target = census.income_bracket
feature_data = census.drop( 'income_bracket', axis = 1)

In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(feature_data)
X = pd.DataFrame(X, columns = columns[:-1])

### Perform a Train Test Split on the Data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = 101)

### Create the Feature Columns for tf.esitmator

In [9]:
feat_col = []
for feat in columns[:-1]:
    feat_col.append(tf.feature_column.numeric_column(feat))

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [10]:
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, 
                                                y = y_train,
                                                batch_size = 200,
                                                num_epochs = None,
                                                shuffle = True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [11]:
model = tf.estimator.DNNClassifier(feature_columns=feat_col, hidden_units = [13,13,13])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_tf_random_seed': None, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_master': '', '_session_config': None, '_service': None, '_num_worker_replicas': 1, '_model_dir': 'C:\\Users\\Amin\\AppData\\Local\\Temp\\tmpvm642n_2', '_global_id_in_cluster': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001AF629CD400>, '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_evaluation_master': '', '_is_chief': True, '_task_id': 0}


** Train your model on the data, for at least 5000 steps. **

In [12]:
model.train(input_fn=input_func, steps=100000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Amin\AppData\Local\Temp\tmpvm642n_2\model.ckpt.
INFO:tensorflow:step = 1, loss = 133.76636
INFO:tensorflow:global_step/sec: 77.7054
INFO:tensorflow:step = 101, loss = 61.261147 (1.289 sec)
INFO:tensorflow:global_step/sec: 98.8908
INFO:tensorflow:step = 201, loss = 61.62831 (1.012 sec)
INFO:tensorflow:global_step/sec: 105.745
INFO:tensorflow:step = 301, loss = 59.981277 (0.945 sec)
INFO:tensorflow:global_step/sec: 104.968
INFO:tensorflow:step = 401, loss = 79.94229 (0.953 sec)
INFO:tensorflow:global_step/sec: 121.791
INFO:tensorflow:step = 501, loss = 63.172394 (0.821 sec)
INFO:tensorflow:global_step/sec: 124.522
INFO:tensorflow:step = 601, loss = 87.84655 (0.803 sec)
INFO:tensorflow:global_step/sec:

INFO:tensorflow:global_step/sec: 113.234
INFO:tensorflow:step = 8101, loss = 65.57392 (0.883 sec)
INFO:tensorflow:global_step/sec: 122.238
INFO:tensorflow:step = 8201, loss = 67.74327 (0.818 sec)
INFO:tensorflow:global_step/sec: 111.156
INFO:tensorflow:step = 8301, loss = 55.985813 (0.920 sec)
INFO:tensorflow:global_step/sec: 114.663
INFO:tensorflow:step = 8401, loss = 63.756943 (0.852 sec)
INFO:tensorflow:global_step/sec: 105.857
INFO:tensorflow:step = 8501, loss = 67.157166 (0.948 sec)
INFO:tensorflow:global_step/sec: 121.421
INFO:tensorflow:step = 8601, loss = 62.75387 (0.821 sec)
INFO:tensorflow:global_step/sec: 113.234
INFO:tensorflow:step = 8701, loss = 74.38392 (0.883 sec)
INFO:tensorflow:global_step/sec: 112.723
INFO:tensorflow:step = 8801, loss = 64.037094 (0.887 sec)
INFO:tensorflow:global_step/sec: 111.404
INFO:tensorflow:step = 8901, loss = 56.966446 (0.898 sec)
INFO:tensorflow:global_step/sec: 117.013
INFO:tensorflow:step = 9001, loss = 67.91792 (0.854 sec)
INFO:tensorflow

INFO:tensorflow:global_step/sec: 100.684
INFO:tensorflow:step = 16401, loss = 55.97519 (0.993 sec)
INFO:tensorflow:global_step/sec: 120.324
INFO:tensorflow:step = 16501, loss = 58.489532 (0.831 sec)
INFO:tensorflow:global_step/sec: 107.393
INFO:tensorflow:step = 16601, loss = 73.0815 (0.931 sec)
INFO:tensorflow:global_step/sec: 111.528
INFO:tensorflow:step = 16701, loss = 69.23442 (0.897 sec)
INFO:tensorflow:global_step/sec: 111.342
INFO:tensorflow:step = 16801, loss = 52.41383 (0.900 sec)
INFO:tensorflow:global_step/sec: 91.0517
INFO:tensorflow:step = 16901, loss = 67.20079 (1.097 sec)
INFO:tensorflow:global_step/sec: 102.861
INFO:tensorflow:step = 17001, loss = 63.608955 (0.972 sec)
INFO:tensorflow:global_step/sec: 111.28
INFO:tensorflow:step = 17101, loss = 60.375343 (0.899 sec)
INFO:tensorflow:global_step/sec: 104.419
INFO:tensorflow:step = 17201, loss = 64.93997 (0.958 sec)
INFO:tensorflow:global_step/sec: 107.163
INFO:tensorflow:step = 17301, loss = 48.990707 (0.934 sec)
INFO:ten

INFO:tensorflow:global_step/sec: 113.879
INFO:tensorflow:step = 24701, loss = 70.404564 (0.878 sec)
INFO:tensorflow:global_step/sec: 98.5008
INFO:tensorflow:step = 24801, loss = 63.377506 (1.015 sec)
INFO:tensorflow:global_step/sec: 116.332
INFO:tensorflow:step = 24901, loss = 53.454742 (0.860 sec)
INFO:tensorflow:global_step/sec: 110.054
INFO:tensorflow:step = 25001, loss = 62.3445 (0.909 sec)
INFO:tensorflow:global_step/sec: 109.093
INFO:tensorflow:step = 25101, loss = 59.168636 (0.917 sec)
INFO:tensorflow:global_step/sec: 113.491
INFO:tensorflow:step = 25201, loss = 57.29492 (0.881 sec)
INFO:tensorflow:global_step/sec: 111.156
INFO:tensorflow:step = 25301, loss = 64.28143 (0.900 sec)
INFO:tensorflow:global_step/sec: 104.803
INFO:tensorflow:step = 25401, loss = 52.09504 (0.954 sec)
INFO:tensorflow:global_step/sec: 114.993
INFO:tensorflow:step = 25501, loss = 57.33363 (0.870 sec)
INFO:tensorflow:global_step/sec: 108.619
INFO:tensorflow:step = 25601, loss = 54.806793 (0.921 sec)
INFO:t

INFO:tensorflow:global_step/sec: 115.793
INFO:tensorflow:step = 33001, loss = 52.875595 (0.861 sec)
INFO:tensorflow:global_step/sec: 117.633
INFO:tensorflow:step = 33101, loss = 55.18711 (0.850 sec)
INFO:tensorflow:global_step/sec: 117.633
INFO:tensorflow:step = 33201, loss = 57.766808 (0.850 sec)
INFO:tensorflow:global_step/sec: 113.75
INFO:tensorflow:step = 33301, loss = 68.34679 (0.880 sec)
INFO:tensorflow:global_step/sec: 114.401
INFO:tensorflow:step = 33401, loss = 47.465023 (0.874 sec)
INFO:tensorflow:global_step/sec: 117.564
INFO:tensorflow:step = 33501, loss = 55.005997 (0.851 sec)
INFO:tensorflow:global_step/sec: 115.126
INFO:tensorflow:step = 33601, loss = 56.783985 (0.869 sec)
INFO:tensorflow:global_step/sec: 115.592
INFO:tensorflow:step = 33701, loss = 61.672066 (0.865 sec)
INFO:tensorflow:global_step/sec: 123.982
INFO:tensorflow:step = 33801, loss = 61.47661 (0.807 sec)
INFO:tensorflow:global_step/sec: 119.819
INFO:tensorflow:step = 33901, loss = 61.222637 (0.835 sec)
INFO

...

INFO:tensorflow:global_step/sec: 102.702
INFO:tensorflow:step = 66201, loss = 58.29658 (0.973 sec)
INFO:tensorflow:global_step/sec: 119.533
INFO:tensorflow:step = 66301, loss = 58.76255 (0.855 sec)
INFO:tensorflow:global_step/sec: 126.173
INFO:tensorflow:step = 66401, loss = 67.32442 (0.775 sec)
INFO:tensorflow:Saving checkpoints for 66437 into C:\Users\Amin\AppData\Local\Temp\tmpvm642n_2\model.ckpt.
INFO:tensorflow:global_step/sec: 90.4336
INFO:tensorflow:step = 66501, loss = 53.502678 (1.106 sec)
INFO:tensorflow:global_step/sec: 103.5
INFO:tensorflow:step = 66601, loss = 61.785084 (0.966 sec)
INFO:tensorflow:global_step/sec: 107.74
INFO:tensorflow:step = 66701, loss = 56.28946 (0.928 sec)
INFO:tensorflow:global_step/sec: 109.511
INFO:tensorflow:step = 66801, loss = 50.789192 (0.913 sec)
INFO:tensorflow:global_step/sec: 113.75
INFO:tensorflow:step = 66901, loss = 53.786793 (0.880 sec)
INFO:tensorflow:global_step/sec: 109.571
INFO:tensorflow:step = 67001, loss = 73.31593 (0.912 sec)
IN

INFO:tensorflow:global_step/sec: 119.034
INFO:tensorflow:step = 74401, loss = 70.864845 (0.840 sec)
INFO:tensorflow:global_step/sec: 119.676
INFO:tensorflow:step = 74501, loss = 73.610916 (0.836 sec)
INFO:tensorflow:global_step/sec: 112.028
INFO:tensorflow:step = 74601, loss = 70.02548 (0.893 sec)
INFO:tensorflow:global_step/sec: 107.22
INFO:tensorflow:step = 74701, loss = 63.067673 (0.933 sec)
INFO:tensorflow:global_step/sec: 117.98
INFO:tensorflow:step = 74801, loss = 61.3368 (0.848 sec)
INFO:tensorflow:global_step/sec: 113.042
INFO:tensorflow:step = 74901, loss = 66.862946 (0.885 sec)
INFO:tensorflow:global_step/sec: 117.911
INFO:tensorflow:step = 75001, loss = 55.54962 (0.848 sec)
INFO:tensorflow:global_step/sec: 115.86
INFO:tensorflow:step = 75101, loss = 65.08809 (0.863 sec)
INFO:tensorflow:global_step/sec: 112.343
INFO:tensorflow:step = 75201, loss = 55.197304 (0.890 sec)
INFO:tensorflow:global_step/sec: 109.153
INFO:tensorflow:step = 75301, loss = 58.950302 (0.916 sec)
INFO:ten

INFO:tensorflow:global_step/sec: 107.393
INFO:tensorflow:step = 82701, loss = 52.635025 (0.931 sec)
INFO:tensorflow:global_step/sec: 109.571
INFO:tensorflow:step = 82801, loss = 62.78782 (0.913 sec)
INFO:tensorflow:global_step/sec: 101.917
INFO:tensorflow:step = 82901, loss = 56.11391 (0.981 sec)
INFO:tensorflow:global_step/sec: 108.56
INFO:tensorflow:step = 83001, loss = 73.50932 (0.921 sec)
INFO:tensorflow:global_step/sec: 108.031
INFO:tensorflow:step = 83101, loss = 53.313236 (0.925 sec)
INFO:tensorflow:global_step/sec: 110.297
INFO:tensorflow:step = 83201, loss = 51.987152 (0.907 sec)
INFO:tensorflow:global_step/sec: 97.6825
INFO:tensorflow:step = 83301, loss = 61.162445 (1.024 sec)
INFO:tensorflow:global_step/sec: 100.786
INFO:tensorflow:step = 83401, loss = 62.794815 (0.993 sec)
INFO:tensorflow:global_step/sec: 107.451
INFO:tensorflow:step = 83501, loss = 62.95381 (0.930 sec)
INFO:tensorflow:global_step/sec: 109.873
INFO:tensorflow:step = 83601, loss = 60.172585 (0.910 sec)
INFO:

INFO:tensorflow:global_step/sec: 109.451
INFO:tensorflow:step = 91001, loss = 52.65379 (0.913 sec)
INFO:tensorflow:global_step/sec: 124.522
INFO:tensorflow:step = 91101, loss = 53.04306 (0.804 sec)
INFO:tensorflow:global_step/sec: 105.411
INFO:tensorflow:step = 91201, loss = 57.0095 (0.961 sec)
INFO:tensorflow:global_step/sec: 109.153
INFO:tensorflow:step = 91301, loss = 46.09204 (0.904 sec)
INFO:tensorflow:global_step/sec: 106.364
INFO:tensorflow:step = 91401, loss = 71.04994 (0.950 sec)
INFO:tensorflow:global_step/sec: 126.975
INFO:tensorflow:step = 91501, loss = 55.557335 (0.778 sec)
INFO:tensorflow:global_step/sec: 114.074
INFO:tensorflow:step = 91601, loss = 55.92576 (0.877 sec)
INFO:tensorflow:global_step/sec: 108.266
INFO:tensorflow:step = 91701, loss = 67.814316 (0.924 sec)
INFO:tensorflow:global_step/sec: 101.4
INFO:tensorflow:step = 91801, loss = 63.083992 (0.986 sec)
INFO:tensorflow:global_step/sec: 111.342
INFO:tensorflow:step = 91901, loss = 58.26156 (0.898 sec)
INFO:tenso

INFO:tensorflow:global_step/sec: 106.591
INFO:tensorflow:step = 99301, loss = 57.309517 (0.939 sec)
INFO:tensorflow:global_step/sec: 106.308
INFO:tensorflow:step = 99401, loss = 47.942078 (0.940 sec)
INFO:tensorflow:global_step/sec: 100.939
INFO:tensorflow:step = 99501, loss = 58.08516 (0.991 sec)
INFO:tensorflow:global_step/sec: 97.5871
INFO:tensorflow:step = 99601, loss = 64.496185 (1.043 sec)
INFO:tensorflow:global_step/sec: 114.401
INFO:tensorflow:step = 99701, loss = 58.871357 (0.856 sec)
INFO:tensorflow:global_step/sec: 108.383
INFO:tensorflow:step = 99801, loss = 61.3555 (0.923 sec)
INFO:tensorflow:global_step/sec: 104.201
INFO:tensorflow:step = 99901, loss = 76.165634 (0.960 sec)
INFO:tensorflow:Saving checkpoints for 100000 into C:\Users\Amin\AppData\Local\Temp\tmpvm642n_2\model.ckpt.
INFO:tensorflow:Loss for final step: 68.56163.


### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [13]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      y=y_test,
      batch_size=len(X_test),
      shuffle=False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [14]:
predictions = model.predict(eval_input_func) # it will create a generator
y_pred = []
for pred in predictions:
    y_pred.append(pred['class_ids'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Amin\AppData\Local\Temp\tmpvm642n_2\model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Each item in your list will look like this: **

In [15]:
model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-11-05:54:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Amin\AppData\Local\Temp\tmpvm642n_2\model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-11-05:54:44
INFO:tensorflow:Saving dict for global step 100000: accuracy = 0.85208315, accuracy_baseline = 0.7611833, auc = 0.90326387, auc_precision_recall = 0.77046305, average_loss = 0.3230091, global_step = 100000, label/mean = 0.23881666, loss = 3155.476, prediction/mean = 0.24092846


{'accuracy': 0.85208315,
 'accuracy_baseline': 0.7611833,
 'auc': 0.90326387,
 'auc_precision_recall': 0.77046305,
 'average_loss': 0.3230091,
 'global_step': 100000,
 'label/mean': 0.23881666,
 'loss': 3155.476,
 'prediction/mean': 0.24092846}

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [16]:
results = classification_report(y_test, np.array(y_pred),
                                target_names=data.income_bracket.unique())

In [17]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score

In [18]:
precision,recall,fscore,_= score(y_test, np.array(y_pred), average = 'macro')
print ('Precision : {}'.format(precision))
print ('Recall    : {}'.format(recall))
print ('F-score   : {}'.format(fscore))
print('AUC   :{}'.format(roc_auc_score(y_test, np.array(y_pred))))

Precision : 0.8045398886470834
Recall    : 0.7703221800455471
F-score   : 0.7848853065677406
AUC   :0.7703221800455471


In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, np.array(y_pred)))

             precision    recall  f1-score   support

          0       0.88      0.93      0.91      7436
          1       0.72      0.61      0.66      2333

avg / total       0.85      0.85      0.85      9769

